In [237]:
import numpy as np
import scipy.linalg as slg

In [434]:
def test_multiplication(n=100,m=10,drange=(2,20)):
    test_multiplication_checks = []
    for _ in range(n):
        #generate a random SymTry matrix
        dim = np.random.randint(*drange)
        diagonal = np.random.randint(*drange,size=dim)
        side_diagonal = np.random.randint(*drange,size=dim-1)
        symtri = SymTri(diagonal,side_diagonal)
        for _ in range(m):
            random_matrix = np.random.randint(*drange,size=(dim,dim))
            random_vector = np.random.randint(*drange,size=(1,dim))[0]
            test_multiplication_checks.append(np.array_equal(symtri * random_matrix, symtri.elements @ random_matrix))
    print(all(test_multiplication_checks))

In [697]:
def test_decomposition(n=300,drange=(2,50)):
    test_qr_checks = []
    for _ in range(n):
        #generate a random SymTri matrix
        dim = np.random.randint(*drange)
        diagonal = np.random.randint(*drange,size=dim)
        side_diagonal = np.random.randint(*drange,size=dim-1)
        symtri = SymTri(diagonal,side_diagonal)
        Q,R = symtri.qr()
        test_qr_checks.append(np.allclose(symtri.elements,Q@R,rtol=1e-10,atol=1e-10))
    print(all(test_qr_checks)) 

In [792]:
def test_eigen(n=100,drange=(2,20)):
    test_eigen_checks = []
    for _ in range(n):
        #generate a random SymTry matrix
        dim = np.random.randint(*drange)
        diagonal = np.random.randint(*drange,size=dim)
        side_diagonal = np.random.randint(*drange,size=dim-1)
        symtri = SymTri(diagonal,side_diagonal)
        
        gt = sorted(np.linalg.eig(symtri.elements)[0])
        cc = sorted(symtri.eigen()[0])
        
        test_eigen_checks.append(np.allclose(gt,cc,atol=1e-1))
    print(all(test_eigen_checks))

In [793]:
test_eigen()

True


In [783]:
class SymTri:
    def __init__(self, main_diagonal,side_diagonal):
        assert len(main_diagonal)-1 == len(side_diagonal), "The side diagonal must be one element smaller than the main one!"
        first_row = [main_diagonal[0]]
        size = len(main_diagonal)
        self.main_diagonal = main_diagonal
        self.side_diagonal = side_diagonal
        self.reset_elements()
            
    def reset_elements(self):
        self.elements = np.diag(self.main_diagonal) + np.diag(self.side_diagonal,k=1) + np.diag(self.side_diagonal,k=-1)
        
    def get_index(self, row, column):
        if abs(column-row) > 1:
            return 0
        elif column == row:
            return self.main_diagonal[column]
        else:
            return self.side_diagonal[min(row,column)]
        
    def set_index(self, row, column, value, mirror=True):
        assert abs(column-row) <= 1, "Cannot set a value that would make the matrix not Tridiagonal!"
        if abs(column-row) == 1:
            assert mirror, "mirror=True must be set in order to manipulate the side diagonal!"
        if column == row:
            self.main_diagonal[column] = value
        elif abs(column-row) == 1 and mirror:
            self.side_diagonal[min(column,row)] = value
        self.reset_elements()
        
    def first_index(self):
        return 0,0
    
    def last_index(self):
        last = len(self.main_diagonal)-1
        return last, last
        
    def mul_vector_right(self, other):
        s = self.side_diagonal
        m = self.main_diagonal
        return np.array([
            (s[i-1]*other[i-1] if i-1>=0 else 0) + m[i]*other[i] + (s[i]*other[i+1] if i+1 < len(other) else 0)
            for i,_ in enumerate(other)
        ])
        
    def __mul__(self,other):
        s = self.side_diagonal
        m = self.main_diagonal
        if isinstance(other, (list,tuple)):
            return self.mul_vector_right(other)
        elif isinstance(other,np.ndarray):
            newmatrix = np.zeros((len(other),len(other)))
            for i in range(len(other)):
                column = other[:,i]
                newmatrix[:,i] = self.mul_vector_right(column)
                
            return newmatrix
        
    def qr(self):
        Q = np.eye(len(self.main_diagonal))
        R = self.elements.copy()
        for i in range(len(self.side_diagonal)):
            G = Givens(R,i)
            R = G.elements @ R
            R[np.abs(R) < 1e-12] = 0
            Q = G.elements @ Q
            
        return Q.T, R
    
    def eigen(self, max_iterations=1000, tol=1e-10):
        Q, R = self.qr()
        A = R @ Q
        eigen = R.diagonal()
        evect = Q.copy()
        for _ in range(max_iterations):
            A_new = SymTri(A.diagonal(),A.diagonal(offset=1))
            Q_new, R_new = A_new.qr()
            new_eigen = R_new.diagonal()
            if np.allclose(new_eigen,eigen,atol=tol):
                break
            A = R_new @ Q_new
            eigen = new_eigen
            evect = evect @ Q_new
            
        
        return eigen, evect
        

In [754]:
class Givens:
    def __init__(self, matrix, position):
        self.matrix = matrix
        self.position = position
        self.dim = len(matrix)
        self.theta = self.calculate_theta()
        c = np.cos(self.theta)
        s = np.sin(self.theta)
        self.rotation_matrix = np.array([[c,-s],[s,c]])
        identity_with_rotation = np.eye(self.dim)
        identity_with_rotation[position:position+2,position:position+2] = self.rotation_matrix
        self.elements = identity_with_rotation
            
    def calculate_theta(self):
        d = self.matrix.diagonal()
        s = self.matrix.diagonal(offset=-1)
        theta = np.arctan(-s[self.position]/d[self.position])
        return theta

In [755]:
a = SymTri([1,9,3,4,3,2],[1,2,3,4,5])

In [760]:
a.qr()

(array([[ 0.70710678, -0.66666667,  0.14561734, -0.01142859,  0.07679368,
          0.16829517],
        [ 0.70710678,  0.66666667, -0.14561734,  0.01142859, -0.07679368,
         -0.16829517],
        [ 0.        ,  0.33333333,  0.58246937, -0.04571437,  0.30717472,
          0.67318066],
        [ 0.        ,  0.        ,  0.78633365,  0.03809531, -0.25597894,
         -0.56098389],
        [ 0.        ,  0.        ,  0.        ,  0.99809705,  0.02559789,
          0.05609839],
        [ 0.        ,  0.        ,  0.        ,  0.        , -0.90976298,
          0.41512808]]),
 array([[ 1.41421356,  7.07106781,  1.41421356,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  6.        ,  2.33333333,  1.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  3.81517438,  4.89274272,  3.1453346 ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  4.00762632,  3.14667237,
          4.99048524],
        [ 0.        ,  0.   